In [1]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
# from transformers import TFAutoModelForSequenceClassification
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Normal BERT test train split (ktrain does not need to do the tokenizer method of splitting and using regression)

In [4]:
# # read the dataset
# df = pd.read_csv('/content/gdrive/MyDrive/Movie Review/Fine_Tuning_Set/IMDB Dataset.csv')
 
# # get the locations
# X = df.iloc[:, :-1]
# y = df.iloc[:, -1]
 
# # split the dataset
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.05, random_state=0)

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/Movie Review/Fine_Tuning_Set/IMDB Dataset.csv')

df.shape
print(df.tail(20))

# get the locations
a = df.iloc[1:]
# y = df.iloc[:, -1]
 
# split the dataset
data_train, data_test = train_test_split(
    a, test_size=0.05, random_state=0)

                                                  review sentiment
49980  A stunning film of high quality.<br /><br />Ap...  positive
49981  And I repeat, please do not see this movie! Th...  negative
49982  To be hones, I used to like this show and watc...  negative
49983  I loved it, having been a fan of the original ...  positive
49984  Hello it is I Derrick Cannon and I welcome you...  negative
49985  Imaginary Heroes is clearly the best film of t...  positive
49986  This movie is a disgrace to the Major League F...  negative
49987  A remake of Alejandro Amenabar's Abre los Ojos...  negative
49988  When I first tuned in on this morning news, I ...  negative
49989  I got this one a few weeks ago and love it! It...  positive
49990  Lame, lame, lame!!! A 90-minute cringe-fest th...  negative
49991  Les Visiteurs, the first movie about the medie...  negative
49992  John Garfield plays a Marine who is blinded by...  positive
49993  Robert Colomb has two full-time jobs. He's kno...  nega

In [6]:
#Or if test and train data are split manually or if another package is used to split the data, retrieve the data.
# Kaggle's competition on the movie review dataset had already given a test and test split dataset.
# data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype= str)
# data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)

In [7]:
print(data_train)

                                                  review sentiment
49928  I love so much about this movie: the music, th...  positive
42051  My partner and I had never heard of this movie...  positive
8845   Father Hood<br /><br />I can understand why a ...  positive
45734  Radio will have you laughing, crying, feeling....  positive
38683  The Shining starts with Jack Torrance (Jack Ni...  positive
...                                                  ...       ...
21244  This movie tries to be artistic but comes acro...  negative
45892  Can anybody do good CGI films besides Pixar? I...  negative
42614  I watched this movie with my mother when I was...  positive
43568  The director has no clue. I know ... That is t...  negative
2733   "Pitch Black" was a complete shock to me when ...  positive

[47499 rows x 2 columns]


In [8]:
data_test

,review,sentiment
11842,If Ashanti had been a serious attempt at a fil...,negative
19386,It was riveting. I just could not look away. A...,positive
16312,If you are wondering where many of the conspir...,negative
19380,If you have trouble suspending disbelief then ...,positive
36753,toplines this ok comedy about an aging father ...,positive
...,...,...
2050,Well - when the cameo appearance of Jason Mill...,negative
28383,I suppose if you like pure action... you'll fi...,negative
44592,"If, like me, you actively seek out the rarest ...",negative
19826,"During the Civil War, there were many cases of...",negative


In [9]:
data_train.sample(5)

,review,sentiment
40144,"In Cold Mountain, North Colorado, near to the ...",positive
7955,If you're looking for a Hollywood action packe...,positive
30529,"I saw this film when I was 10 or 11 years old,...",positive
23340,I was a kid .. crazy about Michael Jackson. Hi...,positive
13667,I really enjoyed this movie as a young kid. At...,positive


In [10]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


# BERT-Base

In [11]:
# ((X_train, y_train), (X_test, y_test), preproc) are used without the ktrain package

In [12]:
(train, val, preproc) = text.texts_from_df(train_df = data_train, 
                                           text_column = 'review', #change according to the dataset used for fine-tuning
                                           label_columns = 'sentiment', #change according to the dataset used for fine-tuning
                                           val_df = data_test,
                                           maxlen = 512, #reduce to maybe 400 for training time reduction, but not in production
                                           preprocess_mode = 'bert')

['negative', 'positive']
       negative  positive
49928       0.0       1.0
42051       0.0       1.0
8845        0.0       1.0
45734       0.0       1.0
38683       0.0       1.0
['negative', 'positive']
       negative  positive
11842       1.0       0.0
19386       0.0       1.0
16312       1.0       0.0
19380       0.0       1.0
36753       0.0       1.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [13]:
model = ktrain.load_predictor('/content/gdrive/MyDrive/BERT_IMDB_Movie_Flask').model

# re-instantiate Learner and training
learner = ktrain.get_learner(model, train_data=train, val_data=val, batch_size=6)
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
7917/7917 [==============================] - 4321s 544ms/step - loss: 0.1255 - accuracy: 0.9556 - val_loss: 0.1097 - val_accuracy: 0.9644


In [14]:
model = text.text_classifier(name = 'bert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 512
done.


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.save('/content/drive/My Drive/BERT_IMDB_Movie_Flask')

# Uncomment below if a user wants to test the model

In [ ]:
# data = ['This movie was really bad. Acting was also bad.', 
        # 'This movie was really great. I will see it again',
        # 'Another great movie. A must watch']

In [ ]:
# predictor.predict(data)

In [ ]:
# predictor.get_classes()

In [ ]:
# predictor.predict(data, return_proba=True)

In [ ]:
# predictor.predict(data[0])